## Mapping IFCB locations with ipyleaflet

Data contains timestamps, URLs, and lat/lon

In [1]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import os

FILE='/home/joefutrelle/data/neslter/NESLTER_transect_ifcb_locations.csv'
assert os.path.exists(FILE)

df = pd.read_csv(FILE, index_col=0)
df.head()

,pid,latitude,longitude
2018-01-31 17:43:21+00:00,http://ifcb-data.whoi.edu/NESLTER_transect/D20...,41.2125,-70.9224
2018-01-31 18:06:20+00:00,http://ifcb-data.whoi.edu/NESLTER_transect/D20...,41.1975,-70.8792
2018-01-31 18:29:33+00:00,http://ifcb-data.whoi.edu/NESLTER_transect/D20...,41.1932,-70.8755
2018-01-31 18:43:15+00:00,http://ifcb-data.whoi.edu/NESLTER_transect/D20...,41.1987,-70.8848
2018-01-31 19:06:27+00:00,http://ifcb-data.whoi.edu/NESLTER_transect/D20...,41.1972,-70.8812


Generate GeoJSON for each IFCB location, including URL as a property

In [2]:
from ipyleaflet import GeoJSON

def ifcb_point(lat, lon, pid):
    return {
        'type': 'Feature',
        'properties': {
            'URL': '{}.html'.format(pid)
        },
        'geometry': {
            'type': 'Point',
            'coordinates': [lon, lat]
        }
    }

ifcb_points = []

for _, pid, lat, lon in df.itertuples():
    ifcb_points.append(ifcb_point(lat, lon, pid))
    
ifcb_layer = GeoJSON(data={'features': ifcb_points})

Add a click handler that opens the URL in a new browser window

In [3]:
import webbrowser

def click_handler(event=None, id=None, properties=None):
    # the 'properties' keyword gives us access to the clicked point's URL
    webbrowser.open(properties['URL'])

ifcb_layer.on_click(click_handler)

Construct a map and add the IFCB locations layer to it.

Using `Esri.NatGeoWorldMap` basemap. For a list of basemaps see [the source code](https://github.com/jupyter-widgets/ipyleaflet/blob/master/ipyleaflet/basemaps.py).

In [4]:
from ipyleaflet import Map, basemaps, Circle

basemap = basemaps.Esri.NatGeoWorldMap

m = Map(center=(41, -70.6731), zoom=8, basemap=basemap)
m.add_layer(ifcb_layer)
m

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{…